### Live projects

In [40]:
# pageno_array = np.arange(1,202,40)
# pageno_array

array([  1,  41,  81, 121, 161, 201])

In [8]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
from IPython.display import display, HTML
import HTMLParser
import pickle
import sys
from mpi4py import MPI

def load_pickle(fname):
    f = open(fname)
    var = pickle.load(f)
    f.close()
    return var

def dump_pickle(variable,fname):
    f = open(fname, 'w')
    pickle.dump(variable, f)
    f.close()
    return None

pageno_array = np.arange(1,202,40)
# machine_no = 0
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
name = MPI.Get_processor_name() # name of the process
print 'comm,rank,size,name =',comm,rank,size,name

machine_no = rank

for pageno in range(pageno_array[machine_no],pageno_array[machine_no+1]):

    print 'At the beginning of page '+str(pageno)
    
    proj_title_title = []
    proj_title_link = []
    proj_blurb_title = []
    creator_list = []
    number_of_backers = []
    dollar_sign_list = []
    amount_list = []
    proj_location_country = []
    Location = []
    currency = []
    proj_location_shortname = []
    proj_location_state = []
    proj_location_type = []
    goal_amount = []
    big_category_list = []
    small_category_list = []
    deadline_list = []
    
    r = urllib.urlopen('https://www.kickstarter.com/discover/advanced?woe_id=0&sort=magic&seed=2462842&page='+str(pageno+1)).read()
    soup = BeautifulSoup(r, 'html.parser')

    proj_chunk = soup.find_all("div", class_="project-card-content")
    
    for items in proj_chunk:
        
        # === Project blurb ===
        blurb_chunk = items.find_all("p",class_="project-blurb")
        proj_blurb_title.append( blurb_chunk[0].string )
#         sys.exit("Error message")
        
        # === Project descriptions ===
        title_str = HTMLParser.HTMLParser().unescape(str(items).decode('utf-8')) # remove the &quot
#         print title_str
        
        
        tag = BeautifulSoup(title_str, 'html.parser')
        proj_title_title.append( tag.a.string )
        proj_title_link.append( tag.a['href'] )
        creator_list.append( tag.p.string.strip() )
        
        # === Next is to scrap things from each link! ===
        r2 = urllib.urlopen("https://www.kickstarter.com/"+proj_title_link[-1]).read()
        soup2 = BeautifulSoup(r2, 'html.parser')
        title_str2 = HTMLParser.HTMLParser().unescape(str(soup2).decode('utf-8')) # remove the &quot
    
        category_info = re.findall('"category":{(.*?)}', title_str2)
        category_info2 = re.findall('"slug":"(.*?)"', category_info[0])
        
        category_strings = category_info2[0].split('/')
        big_category_list.append(category_strings[0])
        if len(category_strings)>1:
            small_category_list.append(category_strings[1])
        else:
            small_category_list.append("N/A")
            
        deadline_info = soup2.find_all("time", class_="invisible-if-js js-adjust-time")
        deadline_list.append( deadline_info[3]['datetime'] )
        
        backers_info = re.findall('<b>(.*?) backers</b>', title_str2)
        if len(backers_info)>0:
            ans = re.findall('\d+', backers_info[0])
            ans2 = ''.join(ans)
            #re.findall('\d+', s)
            try:
                number_of_backers.append( int(ans2) )
            except:
                number_of_backers.append( 'NaN' )
        else:
            try:
                backers_info = re.findall('data-backers-count="(.*?)"', title_str2)
                number_of_backers.append( int(backers_info[0].replace(",", "")) )
            except:
                print 'error retrieving backers_info'
                print backers_info
                print proj_title_link[-1]
                print '-----------------'
            
        
        amount_list.append( re.findall('"pledged":(.*?),', title_str2)[0] )
        dollar_sign_list.append( re.findall('"currency":"(.*?)"', title_str2)[0] )
        
        goal_info = re.findall('pledged of <span class="money">(.*?)</span>', title_str2)
        pledged_amount_number = ''.join( re.findall('\d+', goal_info[0]) )
        
        
        goal_amount.append( pledged_amount_number )
        
        country_info = re.findall('"country":"(.*?)",', title_str2)
        proj_location_country.append(country_info[0])
        
        location_info = re.findall('"short_name":"(.*?)",', title_str2)
        proj_location_shortname.append( location_info[0] )
        
        state_info = proj_location_shortname[-1].split(',')
        proj_location_state.append( state_info[-1] )
        
        type_info = re.findall('"type":"(.*?)","is_root"', title_str2)
        proj_location_type.append( type_info[0] )

    try:
        df = pd.DataFrame({'Project title':proj_title_title, 'Project Link':proj_title_link,\
                           'Project blurb':proj_blurb_title, 'Creator':creator_list,'Deadline':deadline_list,\
                           'Category':big_category_list, 'Sub-category':small_category_list,\
                           'Location':proj_location_shortname, 'Country': proj_location_country,\
                          'State': proj_location_state, 'Type':proj_location_type, \
                           'number_of_backers':number_of_backers,\
                          'Dollar sign': dollar_sign_list, 'Amount':amount_list, 'Goal':goal_amount})
        display(df.head())
    except:
        print 'Page #'+str(pageno+1)+' has an exception.'
        continue
    #     break

    if pageno==pageno_array[machine_no]:
        df2 = df
    else:
        df2 = pd.concat([df2,df])


        
        
# dump_pickle(df2,'live_project_df2_'+str(machine_no)+'.pckl')        

df2.to_csv('live_project_df2_'+str(machine_no)+'.csv',encoding='utf-8')

In [10]:
# df3 = pd.read_csv('live_project_df2_'+str(machine_no)+'.csv')
# df3

,Unnamed: 0,Amount,Category,Country,Creator,Deadline,Dollar sign,Goal,Location,Project Link,Project blurb,Project title,State,Sub-category,Type,number_of_backers
0,0,56275.00,design,US,LetterformArchive,2017-04-28T19:44:36-04:00,USD,50000,"San Francisco, CA",/projects/letterformarchive/w-a-dwiggins-a-lif...,\nWritten & designed by Bruce Kennett. The fir...,W. A. Dwiggins: A Life in Design,CA,graphic design,Town,361
1,1,6313.00,film & video,US,Andy Bailey,2017-04-18T00:22:27-04:00,USD,30000,"Portland, OR",/projects/andymation/flip-book-animation-kit?r...,\nThe Flip Book Animation Kit will give you ev...,Flip Book Animation Kit,OR,animation,Town,177
2,2,41321.00,music,US,Eric Townsend,2017-03-31T20:09:04-04:00,USD,10000,"Los Angeles, CA",/projects/946153346/daniel-amos-horrendous-dis...,\nFrom Stunt Records and Solid Rock Records - ...,Daniel Amos - Horrendous Disc (Deluxe Edition),CA,rock,Town,625
3,3,7859.00,art,US,Print League KC,2017-04-07T17:08:21-04:00,USD,9500,"Kansas City, MO",/projects/1332864089/print-league-kc?ref=disco...,\na future Kansas City community printshop loo...,PRINT LEAGUE KC,MO,NaN,Town,137
4,4,20562.00,art,US,Clark Huggins,2017-04-19T16:00:00-04:00,USD,7500,"Providence, RI",/projects/2078704151/reckless-deck-solving-the...,\nA genre-crossing card deck that generates en...,"RECKLESS DECK: Solving the Problem of ""What Sh...",RI,illustration,Town,560
5,5,5539.00,art,US,Dave Dorman,2017-04-01T23:13:14-04:00,USD,2000,"Chicago, IL",/projects/1932088499/captain-nemo-portrait-by-...,\nWe are creating limited edition prints of on...,"""Captain Nemo"" Portrait by Dave Dorman",IL,illustration,Town,55
6,6,15677.00,games,US,Golden Goblin Press,2017-03-31T23:59:00-04:00,USD,7000,"Queens, NY",/projects/golden-goblin-press/cold-warning-a-c...,\nA 1920's Call of Cthulhu 7th edition scenari...,Cold Warning - A Call of Cthulhu 7th Edition A...,NY,tabletop games,County,556
7,7,9561.00,games,US,Ray Wehrs,2017-04-22T12:01:39-04:00,USD,15000,"Redmond, WA",/projects/781219801/dicey-peaks?ref=discovery,\nA push-your-luck adventure tabletop game for...,Dicey Peaks,WA,tabletop games,Town,297
8,8,1218.00,crafts,US,Heidi,2017-04-08T15:37:54-04:00,USD,1000,"Port Townsend, WA",/projects/2086212281/two-sheep-and-a-llama?ref...,\nI'm looking to launch my handspun yarn busin...,Two sheep and a llama.,WA,knitting,Town,50
9,9,151991.69,games,US,Lay Waste Games LLC,2017-04-11T11:00:00-04:00,USD,14000,"Boston, MA",/projects/laywastegames/dragoon-the-rogue-and-...,"\nChoose between the base game, or the expansi...",DRAGOON: The Rogue and Barbarian Expansion + R...,MA,tabletop games,Town,1789


## Dollar-sign converter
http://utf8-chartable.de/unicode-utf8-table.pl?start=8320&number=128&names=-&utf8=string-literal

## Positive and Negative word List
Tutorial: http://nealcaren.web.unc.edu/pizza-twitter-and-apis

Positive List: http://www.unc.edu/~ncaren/haphazard/positive.txt        

Negative List: http://www.unc.edu/~ncaren/haphazard/negative.txt